### 4.1 Computing the parity of a Word

How would you compute the parity of a very large number of 64-bit words?

Parity of binary words is when the number of 1s is odd, and otherwise 0. 
ex. 1011 is 1 
ex.  1001 is 0

In [3]:
# The following is the my own attempt at the solution
def parity(x):
  '''Computes the the parity of a very large x'''
  count = 0
  while x != 0:
    count += x & 1
    x >>= 1
  if (count % 2 == 0):
    return 0
  return 1

"""-----Tests--------"""
print(parity(0b011))  # 0
print(parity(0b111))  # 1

0
1


#### Solutions:
---
##### Brute Force:
Iteratively tests each bit. Can store the number mod 2 with `^=`
Time complexity is `O(n)` where n is size of word. 
```python
  def parity(x):
    result = 0
    while x:
      result ^= x & 1
      x >>= l
    return result
```

##### Bit-fiddling Trick 
This trick involves iteratively counting the 1s by deleting the next lowest 1 bit 

**Note: `x&(x-1)` equals x with the lowest set bit erased**
This improves the *best* and *average* runtime to `O(k)` where k = number of 1s.


**Note: `x&~(x-1)` isolates the lowest bit**
```python
  def parity(x):
    result = 0
    while x:
      result ^= 1
      x &= x - 1 # Drops the -lowest set bit of x
    return result
```
##### Hasing Results
We can hash a smaller subset of word sizes on what the parity is and compute parites of multiple bits 
at a time. An instance of this would be to use 4 references to 16bit words to compute the parity of 
a 64 bit hash. We use a *mask_size* and a *bit_mask* to find the hash-key. 

This improves the time complexity to `O(n/L)` where
- *n* is the width of the word 
- *L* is the width of the cache
```python
def parity(x):
  MASK-SIZE = 16
  BIT-SIZE = 0xFFFF
  return (PRECOMPUTED_PARITY[x >> (3 * MASK-SIZE)] ^
          PRECOMPUTED_PARITY[(x >> (2 * MASK-SIZE)) & BIT_MASK] ^
          PRECOMPUTED_PARITY[(x >> MASK-SIZE) & BIT_MASK] ^ 
          PRECOMPUTED_PARITY[x & BIT_MASK])
```

##### XOR
**Note that the XOR between two words has the same parity.**
This improves the runtime to `O(log n)`
```python
def parity(x):
  x ^= x >> 32
  x ^= x >> 16
  x ^= x >> 8
  x ^= x >> 4
  x ^= x >> 2
  x ^= x >> 1
  return x & 0x1
```

#### 4.2 Swap Bits

Implement code that takes as input a 64-bit integer and swaps the bits at indices i and j 


In [4]:
# Attempt
def swap(x, i, j):
  '''(binary, num, num) => binary
      x = binary word that needs swaping 
      i = index 1 
      j = index 2 
  '''
  BIT_MASK1 = 0x1 << i
  BIT_MASK2 = 0x1 << j
  if (x & BIT_MASK1 == x & BIT_MASK2):
    return x
  # XOR swaps bits (0 ^ 1 = 1  1 ^ 1 = 0)
  return x ^ (BIT_MASK1 | BIT_MASK2)

print(bin(swap(0b01010, 1, 2))) # 0b1100

# The runtime is O(1)

0b1100


#### 4.3 Reverse Bits

Write a program that takes 64 bit unsigned bits and returns the integers consisting of the bits in reverse order

In [5]:
# Attempted solution
def reverse(x):
  '''Returns bits in reverse'''
  new_x = 0
  while x != 0:
    new_x <<= 1
    new_x |= (x & 1)
    x >>= 1
  return new_x

# Brute force solution takes O(n) time

print(bin(reverse(0b100111))) # 0b111001


0b111001


#### Solutions:
---
##### Lookup Table:
Similar to question 4.3 we can create a lookup table where `x = y3, y2, y1, y0 => y0, y1, y2, y3` and
`y_n` can be found through a lookup table. Time complexity is `O(n/L)`. 
Note that `PRECOMPUTED_REVERSE` is the lookup table of size `2^16`
```python
def reverse-bits (x) :
  MASK_SIZE = 16
  BIT_SIZE = 0xFFFF
return (PRECOMPUTED_REVERSE[x & BIT_MASK] << (3 * MASK_SIZE) |
        PRECOMPUTED_REVERSE[(x >> MASK_SIZE) & BIT_MASK] <<
        (2 * MASK_SIZE) |
        PRECOMPUTED_REVERSE [(X >> (2 * MASK_SIZE)) & BIT_MASK] << MASK_SIZE |
        PRECOMPUTED_REVERSE[(x >> (3 * MASK_SIZE)) & BIT_MASK)
```

#### 4.4 Find a Closest Integer with the same Weight

The weight of a nonnegative integer `x` is the number of bits in its binary representation that is equal to 1

Write a program which takes as input a nonnegative integer x and returns a number `y` which is not
equal to `x`, but has the same weight as `x` and their difference, `| y - x |`,
is as small as possible. You can assume `x` is not 0, or all 1s. For example, 
if `x = 6`, you should return 5. You can assume the integer fits in 64 bits.

In [6]:
def closest_weight(x):
  y = ~x        #invert x 
  first_zero = y & ~(y - 1) #position of first 0
  first_one = x & ~(x - 1) #position of first 1
  return x ^ (first_one | first_zero);

# Time complexity = O(1)
# tests 
print(closest_weight(6))   # 5

5


#### Solutions:
---
##### Heuristic approach:
Note that we simply want to reduce the distance in which bits are swap. Bits have to be swap in order to preserve the weights 
The following solution is similar to my attempt but has a worst case of `O(n)`
```python
def closest_int_sane_bit_count (x) :
  NUll_UNSIGNED_BITS = 64
  for i in range(NUM_UNSIGNED_BITS - 1):
    if (x >> i) & 1 != (x >> (i + 1)) & 1:
      x ^= (1 << i) | (1 << (i + 1)) # Swaps bit-i and bit-(i + 1)
      return x
# Raise error if all bits of x are 0 or 1
raise ValueError('A1l bits are 0 or 1')
```

#### 4.5 Comput X * Y without arithmetical operators 

Write a program to multiple two nonnegative integers with only bitwise operators 

In [7]:
#attempt
def add(x,y):
  carry_in = 0
  carry_out = 0
  binSum = 0
  count = 0
  while (x or y or carry_out):
    lsb1 = x & 1
    lsb2 = y & 1
    carry_out = (lsb1 & lsb2) | (lsb1 & carry_in) | (lsb2 & carry_in)
    binSum |= ((carry_in ^ lsb1) ^ lsb2) << count 
    x >>= 1
    y >>= 1
    carry_in = carry_out 
    count += 1
  return binSum
def multiply(x, y):
  binSum = 0
  for i in range(y): 
    binSum = add(x, binSum)
  return binSum

print(multiply(2, 2))
print(multiply(2, 16))
print(multiply(2, 32))
print(multiply(0, 2))
    
# The following algorithm has a time complexity of O(2^n) which is a brute force solution where n is the lenght of the input bit

4
32
64
0


#### Solutions:
---
##### Shift and Add:
Note that we do not need to do repeated addition but instead multiply by each bit and shift by `2^k` and then add. 
This reduces the time complexity to `O(n^2)`
```python
def multiply(x, y): 
    def add(a, b): 
        # Note k is our mask
        running_sum, carryin, k, temp_a, temp_b = 0, 0, 1, a, b
        while temp_a or temp_b: 
            ak, bk = a & k, b & k 
            carryout = (ak & bk) | (ak & carryin) | (bk & carryin)
            running_sum |= ak ^ bk ^ carryin
            carryin, k, temp_a, temp_b = (carryout << 1, k << 1, temp_a << 1,
                                            temp_b >> 1)
        return running_sum | carryin
   
    running_sum = 0
    while x: # Examines each bit of x.
        if x & 1: 
            running_sum = add(running_sum, y)
        x, y = x >> 1, y << 1
```

#### 4.7 Comput X ^ Y  

Write a program that takes a double x and an integer y and retums x/. You can ignore overflow and underflow.

In [8]:
# Attempt 

def exp(b, x):
    if(x == 0):
        return 1
    if(x == 1):
        return b
    return exp(b, x//2) * exp(b, (x-x//2))

# tests 
print(exp(2, 0))
print(exp(2, 5))
print(exp(1, 4))

# The runtime complexity of the following attempt is O(2^n) where n 
# is the size of x and everything else is constant 

# Brute force example

1
32
1


#### Solutions:
---
##### Recursive interation:
The books example instead of using divide operations uses bits on y in order 
to create a recursive solutions. Observe the following property: 
`x, x^2, (x^2)^2 = x^4, (x^4)^2 = x^8`
 
In bit notation this is as follows:

`x^(1010)_2 = x^(101)_2 * x^(101)_2`

or 

`x^(101)_2 = x^(10)_2 * x

The solution is as follows

```python
def power(x,y):
    result, power = 1.0, y
    if y < 0: 
        power, x = -power, 1.0 / x
    while power:
        if power & 1:
            result *= x 
        x, power = x * x, power >> 1
    return result
```

The time complexity given all else constant is at most twice the index of y's
MSB which is `O(n)`


#### 4.8 Reverse Digits

Write a program which takes an integer and returns the integer corresponding
to the input written in the reverse order.


In [9]:
# Attempt 

def reverse(x):
    digitStr = str(abs(x))
    digitStr = digitStr[::-1]
    if (x < 1):
        return -(int(digitStr))
    return int(digitStr)

# tests 
print(reverse(42))
print(reverse(-314))
print(reverse(0))

# The runtime complexity of this algorithm is O(1) assuming everything is constant
# It is also O(n) if we factor in the cost of reversing the str

24
-413
0


#### Solutions:
---
##### Store the module of 10:
The solution of the book highlights that we do not need to do it in a brute 
force fashion of converting int to a string but instead use %10. 
The time complexity of the following is `O(n)` where n is the number of digits
```python
def reverse(x): 
    result, x_remaining = 0, abs(x)
    while x_remaining:
        result = result * 10 + x_remaining % 10
        x_remaining //= 10
    return -result if x < 0 else result
```

#### 4.11 Rectangle Intersection

This problem is concerted with rectangles whose sides are parallel to the x and y-axis 

Write a program which tests if two rectangles have a nonempty intersection.
If non,empty, return the rectangle formed by their intersection.

In [10]:
# Attempt 

def rectIntersection(R1, R2):
    flag1 = (R1['x'] >= R2['x'] and R1['x'] <= (R2['x'] + R2['width'])) 
    flag2 = (R2['x'] >= R1['x'] and R2['x'] <= (R1['x'] + R1['width']))
    flag3 = (R1['y'] >= R2['y'] and R1['y'] <= (R2['y'] + R2['height']))  
    flag4 = (R2['y'] >= R1['y'] and R2['y'] <= (R2['y'] + R2['height']))
    print(flag1, flag2, flag3, flag4)
    if (not((flag1 or flag2) and (flag3 or flag4))):
        return None
    x, width = (R1['x'], (R2['x'] + R1['width'] - R1['x'])) if (flag1) else (R2['x'], (R1['x'] + R2['width'] - R2['x']))
    y, height = (R1['y'], (R2['y'] + R1['height'] - R1['y'])) if (flag3) else (R2['y'], (R1['y'] + R2['height'] - R2['y']))
    return (x, y, width, height)

# tests
dict1 = {'x': 1,'y': 2,'width': 3,'height': 4}
dict2 = {'x': 5, 'y': 3, 'width': 2, 'height': 4}
dict3 = {'x': 2, 'y': 2, 'width': 2, 'height': 4}
print(rectIntersection(dict1, dict2)) # None
print(rectIntersection(dict1, dict3)) # None

False False False True
None
False True True True
(2, 2, 1, 4)


##### Solutions 
---
##### General Approach 
The highlighted solution is simply to observe the cases in which there are no 
intersections This is by checking the dimensions in which the inclusive ranges 
do not intersect. 

Also not that the intersection creates a rectangle where the included points are the min of origin point, and the max of width and height 

```python
# Useful library for creating object tuples
Rectangle = collections.namedtuple('Rectangle', ('x', 'y', 'width', 'height'))

def intersect_rectangle(R1, R2):
    def is_intersect(R1, R2):
        return (R1.x <= R2.x + R2.width and R1.x + R1.width >= R2.x
                and R1.y <= R2.height and R1.y + R1.height >= R2.y)
    
    if not is_intersect(R1, R2):
        return Rectangle(0, 0, -1, -1) # No intersection
    return Rectangle(
        max(R1.x, R2.x), 
        max(R1.y, R2.y),
        min(R1.x + R1.width, R2.x + R2.width) - max(R1.x, R2.x),
        min(R1.y, + R1.height, R2.y + R2.height) - max(R1.y, R2.y))
```

The runtime for the following equation is `O(1)`